문장의 사전적 정의: 생각이나 감정을 말과 글로 표현할 때 완결된 내용을 나타내는 최소의 단위  
시퀀스란 데이터에 순서를 붙여 하나씩 나열한 것. 특정 위치의 데이터를 가리킬 수 있다. ex) 리스트, 튜플, 레인지, 문자열  
문법을 배워서 인공지능이 문장을 예측할 수 없으니 통계적인 접근방법 선택! -> 많은 데이터가 곧 좋은 결과 ->가장 좋은 인공지능이 RNN(순환신경망)  
통계적인 이라는 뜻은 대체로 \~~하다라고 생각하면 된다  

In [1]:
#첫 시작은 <start>라는 토큰으로 표시. 끝은 <end>라는 토큰으로 표시  
#<start>가 문장의 시작인 입력데이터와 <end>가 문장의 끝인 출력데이터가 필요.
sentence = " 나는 밥을 먹었다 "

source_sentence = "<start>" + sentence
target_sentence = sentence + "<end>"

print("Source 문장:", source_sentence)
print("Target 문장:", target_sentence)

Source 문장: <start> 나는 밥을 먹었다 
Target 문장:  나는 밥을 먹었다 <end>


n-1개의 단어 시퀀스가 주어질 때 n번쨰 단어가 뭐일지 예측하는 확률모델을 언어모델.  
파라미터 θ로 모델링하는 언어모델  
<img src="수식.png" align="left">

n-1까지의 단어 시퀀스가 x_train, n번째가 y_train.

In [2]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os

# 파일을 읽기모드로 열어 봅니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f: # with문을 사용하면 with 블록을 벗어나는 순간 열린 파일 객체 f가 자동으로 close되어 편리하다.
    raw_corpus = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 list 형태로 읽어옵니다.

print(raw_corpus[:9])    # 앞에서부터 10라인만 화면에 출력해 볼까요?

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


화자 표기된 곳: 0, 3, 6  
공백: 2, 5, 9  
쓸모없는 데이터 처리!  
<img src="셰익스피어.png" align="left">  


In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


문장을 쪼개는 것을 토근화라고 한다.  

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


델의 입력이 되는 문장을 소스 문장(Source Sentence)(X_train), 정답 역할을 하게 될 모델의 출력 문장을 타겟 문장(Target Sentence)(y_train).

In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

우리가 가르칠 데이터는 숫자로 전달해야 함으로. 딕셔너리에 데이터를 숫자로 변환.(이 과정을 벡터화), 숫자로 변환된 데이터를 텐서

In [6]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f979361d990>


In [7]:
print(tensor[:3, :10])
#출력값이 0으로 나온 부분은 패딩이 들어간 것으로 길이를 맞추기 위해 넣은것

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [8]:
#텐서 데이터는 모두 정수로 구성. 이 숫자는 tokenizer에 구축된 단어사전의 인덱스
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [9]:
#생성된 텐서를 소스와 타겟으로 분리.
print(tensor.shape)
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(src_input.shape)
print(tgt_input[0])

(24015, 21)
[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
(24015, 20)
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

텐서플로 데이터셋 생성과정  
1.정규표현식을 이용한 corpus생성  
2.tf.keras.preprocessing.text.Tokenizer를 이용해 corpus -> tensor  
3.tf.data.Dataset.from_tensor_slices()를 이용해 corpus 텐서 -> tf.data.Dataset  
위의 과정을 텐서플로의 데이터 전처리라고 한다.

우리가 만들 모델은 tf.keras.Model을 subclassing.  
Embedding레이어 1개, LSTM레이어 2개, Dense레이어 1개로 구성

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
#Embedding 레이어가 단어사전의 인덱스를 해당 인덱스의 워드벡터로 바꿔준다.
#embedding_size는 워드벡터의 차원수. 값이 커질 수록 추상적인 특징을 잡을 수 있지만, 그만큼 충분한 데이터가 필요.
#LSTM레이어의 hidden state의 차원수인 hidden_size도 같은 역할.모델에 얼마나 많은 일꾼을 쓸것인지 결정.


In [12]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)
#model의 input shape가 결정되면서 model.build()가 자동으로 호출됩니다

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[-1.5287522e-04,  3.5295959e-04, -4.9959397e-04, ...,
          4.6026160e-04, -4.4646331e-05, -3.3868867e-04],
        [-1.6363825e-04,  5.7893735e-04, -6.7387172e-04, ...,
          3.0235498e-04, -1.3928095e-04, -5.8676355e-04],
        [ 1.4312154e-04,  6.3640939e-04, -4.9666391e-04, ...,
          2.5546073e-04, -1.8916781e-04, -4.9723283e-04],
        ...,
        [-2.4677252e-03,  5.4253044e-04,  3.4631770e-03, ...,
          4.4178432e-03,  6.9964339e-04, -2.0891894e-03],
        [-2.6790332e-03,  5.0510722e-04,  4.0180958e-03, ...,
          4.9024201e-03,  7.8489422e-04, -2.2059935e-03],
        [-2.8626632e-03,  4.5343651e-04,  4.5104185e-03, ...,
          5.3202952e-03,  8.6191756e-04, -2.2843801e-03]],

       [[-1.5287522e-04,  3.5295959e-04, -4.9959397e-04, ...,
          4.6026160e-04, -4.4646331e-05, -3.3868867e-04],
        [-2.0027682e-04,  3.4450230e-04, -6.9246389e-04, ...,
          6.8491843e-04, -

tensor 출력 shape가 256, 20, 7001이다.  
7001은 dense레이어의 출력 차원.(즉, 7001의 단어가 나올 수 있다, 각 단어의 확률)  
256은 이전 스텝에서 지정한 배치 사이즈입니다. dataset.take(1)를 통해서 1개의 배치를 가져옴.
20은 tf.keras.layers.LSTM(hidden_size, return_sequences=True)로 호출한 LSTM 레이어에서 return_sequences=True에서 자신에게 입력된 시퀀스길이만큼 동일한 길이의 시퀀스를 출력. return_sequences=False일경우 vector 1개만 출력.  
우리 데이터 셋의 len가 20으로 맞춰져있음.

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)
#loss값이 감소하고 있기에 학습이 잘 진행되고 있다.

Epoch 1/30
93/93 [==============================] - 14s 156ms/step - loss: 3.4982
Epoch 2/30
93/93 [==============================] - 15s 157ms/step - loss: 2.8132
Epoch 3/30
93/93 [==============================] - 15s 158ms/step - loss: 2.7286
Epoch 4/30
93/93 [==============================] - 15s 159ms/step - loss: 2.6359
Epoch 5/30
93/93 [==============================] - 15s 161ms/step - loss: 2.5676
Epoch 6/30
93/93 [==============================] - 15s 160ms/step - loss: 2.5170
Epoch 7/30
93/93 [==============================] - 15s 160ms/step - loss: 2.4749
Epoch 8/30
93/93 [==============================] - 15s 160ms/step - loss: 2.4303
Epoch 9/30
93/93 [==============================] - 15s 160ms/step - loss: 2.3850
Epoch 10/30
93/93 [==============================] - 15s 161ms/step - loss: 2.3439
Epoch 11/30
93/93 [==============================] - 15s 160ms/step - loss: 2.3018
Epoch 12/30
93/93 [==============================] - 15s 161ms/step - loss: 2.2599
Epoch 13/30
9

In [15]:
#성능 확인을 위해 작문을 시켜보고, 평가!
#아래 함수는 시작 문장을 바탕으로 작문시작.
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.
#while문을 통해 문장생성.
#텍스트 생성시 타겟문장이 없고, 소스문장이 없다. 그냥 학습시킨 걸 기반으로 만든다.
#init_sentence를 인자로 받아 텐서로 변환.
# <start>
# <start> Hi
# <start> Hi my

In [16]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he is not lolling on the <unk> s face , <end> '

# 프로젝트 시작

In [17]:
import glob # glob를 이용하여 모든 txt 파일을 읽음.
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines() #문장단위로 저장
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me']


In [18]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

In [19]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [32]:
corpus = []
after_corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))

print(len(corpus))        
print(corpus[:10])

for i in range((len(corpus))-1):
    if len(corpus[i].split(' ')) <16:
        after_corpus.append(corpus[i])
corpus = after_corpus

print(len(corpus)) 
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

175749
['<start> hook <end>', '<start> i ve been down so long , it look like up to me <end>', '<start> they look up to me <end>', '<start> i got fake people showin fake love to me <end>', '<start> straight up to my face , straight up to my face <end>', '<start> i ve been down so long , it look like up to me <end>', '<start> they look up to me <end>', '<start> i got fake people showin fake love to me <end>', '<start> straight up to my face , straight up to my face verse <end>', '<start> somethin ain t right when we talkin <end>']
156012


In [33]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 966   3 ...   0   0   0]
 [  2   4  95 ...  10  12   3]
 [  2  38 133 ...   0   0   0]
 ...
 [  2  47  47 ...   0   0   0]
 [  2   3   0 ...   0   0   0]
 [  2   3   0 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f969478d810>


In [34]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,train_size = 0.8)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124809, 14)
Target Train: (124809, 14)


In [35]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [36]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
#Embedding 레이어가 단어사전의 인덱스를 해당 인덱스의 워드벡터로 바꿔준다.
#embedding_size는 워드벡터의 차원수. 값이 커질 수록 추상적인 특징을 잡을 수 있지만, 그만큼 충분한 데이터가 필요.
#LSTM레이어의 hidden state의 차원수인 hidden_size도 같은 역할.모델에 얼마나 많은 일꾼을 쓸것인지 결정.


In [37]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.69407082e-04,  2.34630235e-04,  8.27339463e-05, ...,
         -3.22848471e-04,  1.62889410e-05,  2.24639021e-04],
        [-5.09555975e-04,  3.94151779e-04,  1.80705192e-05, ...,
         -2.57879437e-04, -2.87380885e-06,  5.45676507e-04],
        [-6.14647055e-04,  5.55543229e-04,  1.91670388e-05, ...,
         -3.33313888e-04,  1.72477608e-04,  5.56239625e-04],
        ...,
        [-1.15552742e-03, -9.11077252e-04, -6.47684501e-04, ...,
         -3.42394778e-04, -4.04188177e-04,  4.22295579e-06],
        [-1.22184074e-03, -1.27720379e-03, -9.00728279e-04, ...,
         -1.82931355e-04, -9.12488671e-04,  1.15402312e-04],
        [-1.21427677e-03, -1.63967477e-03, -1.21179968e-03, ...,
         -9.38154626e-05, -1.40621921e-03,  3.24869936e-04]],

       [[-1.69407082e-04,  2.34630235e-04,  8.27339463e-05, ...,
         -3.22848471e-04,  1.62889410e-05,  2.24639021e-04],
        [-3.50807735e-04,  1.72908258e-04,  1

In [38]:
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_2 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_3 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_1 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [39]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)
#loss값이 감소하고 있기에 학습이 잘 진행되고 있다.

Epoch 1/10
609/609 [==============================] - 87s 143ms/step - loss: 3.4139
Epoch 2/10
609/609 [==============================] - 89s 146ms/step - loss: 2.9661
Epoch 3/10
609/609 [==============================] - 90s 148ms/step - loss: 2.7988
Epoch 4/10
609/609 [==============================] - 90s 148ms/step - loss: 2.6700
Epoch 5/10
609/609 [==============================] - 91s 149ms/step - loss: 2.5607
Epoch 6/10
609/609 [==============================] - 91s 149ms/step - loss: 2.4628
Epoch 7/10
609/609 [==============================] - 91s 149ms/step - loss: 2.3730
Epoch 8/10
609/609 [==============================] - 91s 149ms/step - loss: 2.2897
Epoch 9/10
609/609 [==============================] - 91s 149ms/step - loss: 2.2119
Epoch 10/10
609/609 [==============================] - 91s 150ms/step - loss: 2.1387


In [42]:
#성능 확인을 위해 작문을 시켜보고, 평가!
#아래 함수는 시작 문장을 바탕으로 작문시작.
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.
#while문을 통해 문장생성.
#텍스트 생성시 타겟문장이 없고, 소스문장이 없다. 그냥 학습시킨 걸 기반으로 만든다.
#init_sentence를 인자로 받아 텐서로 변환.
# <start>
# <start> Hi
# <start> Hi my

In [43]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , liberian girl <end> '